# 基盤技術開発テクニック練習ノートブック
## 2021年度ver
### 著者
広渡朱莉, 田中大揮，廣田憲幹，中島成太郎，宮崎一希

## このノートブックについて
数式の実装には多くのやり方があります．例えば，行列演算の実装方法は大きく分けて二通りあります．ひとつはfor文によって要素ずつ計算する方法，もうひとつは行列計算ライブラリによって計算する方法です．


このノートブックでは古川研究室でよく使われる行列演算のテクニックをできるだけ網羅してまとめました．また，これらのテクニックを実際に使ってみながら体系的に理解・学習できるようになることを目標に作りました．

修正事項
- argminの説明が演繹的
- jax, einsumの実装がまだ

## 事前知識
このノートブックでも必要最低限の操作は抑えますが，事前知識として以下の本を読むことを推奨します．

[Dropboxにpdf](https://www.dropbox.com/s/7r6cbcux1br1jhb/%E3%82%BC%E3%83%AD%E3%81%8B%E3%82%89%E4%BD%9C%E3%82%8BPython%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E5%85%A5%E9%96%80_pass.pdf?dl=0)があるので手元におくのを推奨します．

- 「ゼロからつくるPython機械学習プログラミング入門」
  - chapter2：Python入門
  - chapter3：数学のおさらい


## 準備
### [Numpy](https://numpy.org/doc/stable/user/index.html)

NumpyはPythonで科学的な計算を行うための基本的なパッケージです．行列計算は機械学習に限らず数値計算の基本であり，Numpyは高速に行列計算を行うことができます．

In [ ]:
# numpyをimportします． as npで短縮してnpという名前で使うのが一般的です．
import numpy as np

In [ ]:
# 5次元の単位行列を作ります．
A = np.eye(5)

# 中身を確認します．
print(A)
# 配列のサイズを確認します． (5, 5)は 5x５の行列になっているという意味です． 
# また， jupyterではセルの最後の行はprint()と括らなくても出力されます．
A.shape

### [Matplotlib](https://)
MatplotlibはPythonの描画パッケージです（描画の実装で悩むのはアルゴリズムの実装と理解にはあまり関係ないので，困ったら知ってる人に質問することをおすすめします）．

In [ ]:
import matplotlib.pyplot as plt

N = 10 # データ数
D = 2 # 次元数

# 二次元データをN個つくります．
X = np.arange(N*D).reshape(N,D)
print(X, X.shape)

In [ ]:
# 散布図(scatter)で描画します． 
## 第一引数にはx座標の値のベクトル．第二引数にはy座標の値のベクトルをいれます．
plt.scatter(X[:,0], X[:,1], marker='x')
[plt.text(X[n,0], X[n,1], "("+str(X[n,0])+","+str(X[n,1])+")") for n in range(N)]
print("")

### 自動微分ライブラリ群
今は迷走期で，FILab内でもこれ使えばいいよ！って言い切れるのはないイメージ．ただ，どれも自動微分としてはちゃんと動くのでどれ使ってもいい．

#### [Pytorch](https://pytorch.org/get-started/locally/)
FILabで自動微分ライブラリとして最初に使われ出した．現ラボメンの半分はこれを使ってるイメージ

特徴:
- MITがメインで開発（はず）
- Pytorchの変数からNumpy変数への変換が他言語と比べて面倒．(torch変数.detach().numpy())．また，変換しないと描画ライブラリで使えない（はず）
- Academicな世界ではよく使われているライブラリらしい
  - GPytorchとか（GPy + torch)の混合ライブラリだったりする
- Sparse行列演算処理が実装済み（ビッグデータに適用可能）
- 自動微分のやり方が下の二つとちと違う

#### [Tensorflow](https://www.tensorflow.org/install?hl=ja)
Sparse行列xNumpyへの親和性を考えて宮崎はこれを使い出した．ラボ内標準はこっちでいい気がしている（宮崎）

特徴:
- Googleがメインで開発（はず）
- Numpyとの相性Not bad
- ビジネスの現場でよく使われているライブラリらしい
  - Tensorflow liteとかある 
- [公式ドキュメント豊富](https://www.tensorflow.org/overview?hl=ja)
- tensorflow2と1でコードが全然違うらしい
- kerasっていうライブラリと一緒になったからなんかネストが一個深い
- Sparse行列演算処理が実装済み（ビッグデータに適用可能）

#### [Jax](https://github.com/google/jax#installation)
速度・軽さが素晴らしいイメージ．　GPUへの適用も簡単にできる（ライブラリ側が勝手に判断してくれるからコード側で書かなくていい）


特徴:
- Googleがメインで開発（はず）．Tensorflowの自動微分の機能とGPUを使う機能だけを取り出してライブラリ化しているみたい．（DLを使わないならぴったり）
- Numpyとの相性Good
- 上の二つのライブラリと比較して新しいライブラリとなっているため，ネットの情報は少なめ
- Windowsには標準対応してないらしい（できる方法は上がっているらしい）
- Sparse行列演算処理が未実装（Issueには上がっていて実装されているらしいからもう少しなはず）

In [1]:
import torch
import tensorflow as tf
import jax,jaxlib

ModuleNotFoundError: No module named 'torch'

### Scipy
一般的な機械学習のライブラリです．

In [ ]:
import scipy

## 目次
0. リシェイプ（reshape)
1. ブロードキャスト（Broadcast）
2. メッシュグリッド（Meshgrid)
3. アーグミン（Argmin)
4. ファンシィインデックス（Fancy index)
5. アインサム（Einsum)

### 0. Reshape
さっきでてきた
> X = np.arange(N*D).reshape(N,D)

これです．

[reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)はデータの形（サイズ）を変更するときに使います．

In [ ]:
# xxx, yyy を適当な整数に変えて実行してください．
#N, D = xxx, yyy
N, D = 8, 2

# NxD次元ベクトルを作ります．
X = np.arange(N*D)
print("Xのshapeは", X.shape,"すなわち", N*D,"次元ベクトル（",N*D,"個要素がある）ということです．")
print(X)
X_m = X.reshape(N,D)
print("\n")
print("これに対しN, D =",N,",",D,"として X.reshape(N,D)とすると", X_m.shape,"の行列になります．")

In [ ]:
print("また，逆にベクトルに戻すには以下のようにします．")

print('X.reshape(-1)', X.reshape(-1))

#### 演習問題0
以上の操作を任意の行列で実践せよ．

### 1. Broadcast
詳しい定義は[上島さんの解説記事](https://www.kamishima.net/mlmpyja/nbayes2/broadcasting.html).
使い方やイメージが欲しいなら[ここ](https://deepage.net/features/numpy-broadcasting.html).

In [ ]:
I = 3
J = 4
A = np.arange(I)[:, None] + 1
print("Aのshapeは",A.shape," で，A自体は",'\n',A, "\n")
B = np.arange(I*J).reshape(I,J)
print("Bのshapeは",B.shape," で，B自体は",'\n', B, "\n")

print("この行列A,Bに対して B-A という演算をすると","\n", B - A,"となります．")

# B - A

行列演算はシェイプが違う行列同士の演算は定義されていません（というかできません）が，Broadcastをサポートしているライブラリでは，ある条件を満たした場合に特殊な演算が行えます（行われてしまいます）．

今回のケース:
A行列（shape:(3,1)）とB行列（shape:(3,4)）はshapeが違うので演算できません． 
Broadcastでは，A行列(3,1)を列方向にコピーしてA_hat(3,4)の行列を作ることで，B - A_hatの演算を行います．

In [ ]:
print("具体的には，以下のような演算をします．")
A_hat = np.tile(A, J)
print("A_hat:","\n",A_hat)
print("B - A_hat:", "\n", B - A_hat,"\n", "先ほどのB-Aの演算と一致します．")

#### 演習問題1-a 
列方向のコピーではなく，行方向のコピーが行われるBroadcastの計算を実装せよ（任意の四則演算）．

In [ ]:
C = np.arange(J)[None, :]
B * C

#### 演習問題1-b 
先ほどは行列（shapeが(I, J)のような変数）に対しての演算であった．3階のテンソルの変数（shapeが （I,J,K））に対して，任意のBroadcastの計算を実装せよ．

In [ ]:
K = 2
D = np.arange(I*J*K).reshape(I,J,K)
E1 = np.arange(I*J).reshape(I,J,1)
E2 = np.arange(I*K).reshape(I,1,K)
print(D-E1,"\n",D-E2)

### 2. Meshgrid

[Meshgrid](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html)は，座標ベクトルを入力すると座標行列を返してくれる関数です．

以下のような軸をx,y軸を想定して，そこにそれぞれ等間隔に点が打たれているとします．

In [ ]:
K = 11
A = np.linspace(-1,1,K)
B = np.linspace(-2,2,K)
plt.plot([A[-1], A[0]], [0,0], color='black', alpha=0.4)
plt.plot([0,0],[B[-1], B[0]], color='black', alpha=0.4)
plt.scatter(A,np.zeros(A.shape), color='orange', marker='x')
plt.scatter(np.zeros(B.shape),B, color='g', marker='x')

この点からそれぞれ垂直線をひくと，以下の図のようなグリッドを描くことができます．

meshgrid関数は以下の図のような格子点（交わっているところ）の座標を取得したいときに使われる関数です．

In [ ]:
plt.scatter(A,np.zeros(A.shape), color='orange', marker='x')
plt.scatter(np.zeros(B.shape),B, color='g', marker='x')
[plt.plot(np.tile(A[i],K), B, color='orange') for i in range(K)]
[plt.plot(A, np.tile(B[i], K),  color='g') for i in range(K)]
print("この緑線とオレンジ線の交点の座標を求めます．")

In [ ]:

XX, YY = np.meshgrid(A,B)

print("二つのベクトルを入力すると二つの行列が返ってきます．")
print("XXのshapeは", XX.shape, 'YYのshapeは', YY.shape, "となります．")
print("XX")
print(XX)
print("YY")
print(YY)

$XX_{ij}$にはx=i,y=-jの交点のx座標が，$YY_{ij}$にはx=i,y=-jの交点のy座標が格納されています．
言い換えると， x=i, y=-jの交点の座標は$( XX_{ij}, YY_{ij} )$ です．

meshgridを使うと，交点の座標の数値は得られました．ここからは，二つの行列を使いやすい形に整形します．
ここでいう使いやすい形とは，いわゆるデータ行列のような形（shapeが(交点の数，次元数)）の二次元配列です．つまり，

$M = \\
[(XX_{11}, YY_{11}),\\(XX_{12}, YY_{12}), \\(XX_{13}, YY_{13}), \\ \cdots, \\(XX_{KK}, YY_{KK}）] $

のような 
$\mathbf{R}^{(K*K)\times 2} = \mathbf{R}^{(P \times 2)},P = K*K$の二次元行列$M$に整形します．
言い換えると，$M = ( \mathbf{m_p} )_{p=1}^{P}, \mathbf{m} \in \mathbb{R}^2$となります． 


まず，XXとYYは (K, K)のような二次元配列である必要ではないので，reshapeを使ってKxKの行列をM次元ベクトルに直します． 
次に，1次元ベクトルxxとyyを横方向に結合するために，reshapeされたxx, yyを(M, 1)の行列に整形して，concatenateします．

In [ ]:
xx = XX.reshape(-1)
yy = YY.reshape(-1)
M = np.concatenate([xx[:,None], yy[:,None]], axis=1)
M

格子点行列Mができました．

#### 演習問題2

以上の操作を任意のベクトルの格子点で実践せよ

#### 補足
格子点を作るには，itertoolsを使う方法もある．（ただし上のやり方とは順番が違って


In [ ]:
import itertools

mesh = np.linspace(-1, 1, K)
Zeta = np.array(list(itertools.product(mesh, repeat=2)))
Zeta

### 3. Argmin, Argmax

>数学において、最大値を与える引数あるいは最大点集合 (argument of the maximum) は関数がその最大値をとる定義域の元全体の成す集合である[note 1]。省略してarg&nbsp;max (もしくは argmax) と書かれる。最大値が函数の出力のうち最も大きいものを指すのと対照に、最大点は最大値を出力する入力の値を指す。
最大点集合は一般に複数の元を含むが、それは有限集合であることも無限集合であることも起こり得るし、空となることもあり得る。

上が一応定義ですが，とりあえず以下の例をみながら理解していきましょう．

例えば，先ほどのような格子点は以下のようなインデックスを持ちます．

In [ ]:
plt.scatter(M[:, 0], M[:, 1], alpha=0.4, marker='D')
[plt.text(M[n, 0], M[n, 1], str(n)) for n in range(K*K)]
print("")

さらに，この空間に以下のような新規点$\mathbf{x}_n$ができたとします．

In [ ]:
np.random.seed(6)
x = np.random.rand(2).reshape(1,2)

plt.scatter(M[:, 0], M[:, 1], alpha=0.4, marker='D')
plt.scatter(x[:, 0], x[:, 1], s=80, color='g', marker='x', label='$\mathbf{x}_n$', linewidths=4)
plt.legend()
# plt.text(X[:, 0]+0.05, X[:, 1], "$\mathbf{x}^*$", color='g', size=25)

$\newcommand{\argmax}{\mathop{\rm arg~max}\limits}$
$\newcommand{\argmin}{\mathop{\rm arg~min}\limits}$

ここでは， $\mathbf{x}_n$とそれに一番近い$\mathbf{m}_{k^*_n}$の点を求めます． 
${k^*_n}$は$n$番目のデータに対して一番近い$\mathbf{m}$のインデックスで勝者ノード番号と呼びます（格子点のインデックス（番号）なのでノード番号とも呼びます）．
格子点は座標の値が固定で非連続なので（このような点を離散点とも呼びます），この勝者ノード番号を求めれば，一番近い点を求めたことになります．

この操作はを数式で書くと，
$k^*_n = \argmin_k D(\mathbf{x}_n, \mathbf{m}_k)$

$D(\mathbf{a},\mathbf{b})$は$\mathbf{a}$と$\mathbf{b}$の距離を返す関数で， $D(\mathbf{a},\mathbf{b})=\|\mathbf{a} - \mathbf{b} \|^2$とすると，

$k^*_n = \argmin_k \|\mathbf{x}_n - \mathbf{m}_k\|^2$ となります．


In [ ]:
print(x.shape, M.shape)
Dist = np.sum((x[:, None, :] - M[None, :, :])**2, axis=2)
print(Dist.shape)

k_star = np.argmin(Dist)

k_star

データ$\mathbf{x}_n$に対応する勝者ノード番号は 75でした．

なので$\mathbf{x}_n$に対応する座標を$\mathbf{z}_n$とおくと

$\mathbf{z}_n := \mathbf{m}_{75}$

ということです．

In [ ]:
plt.scatter(M[:, 0], M[:, 1], alpha=0.4, marker='D', s=10)

z_n = M[k_star] 
plt.text(z_n[0], z_n[1], str(k_star), fontsize=13) 
# plt.text(M[k_star, 0], M[k_star, 1], str(k_star), fontsize=13) 

plt.scatter(z_n[0], z_n[1],  marker='D', s=40, color='g')
plt.scatter(x[:, 0], x[:, 1], s=40, color='g', marker='x', label='$\mathbf{x}_n$', linewidths=2)
print("")

データに一番近い格子点のインデックスが表示され，データと同じ色に着色されているのが確認できたと思います．


では次にデータ１点ではなく，N点で考えます．
データ集合$X = \{ \mathbf{x}_n\}_{n=1}^N$は以下のように生成されます．

In [ ]:
N = 9
X = np.random.rand(N*2).reshape(N,2)
cmap = plt.get_cmap("tab10") 

plt.scatter(M[:, 0], M[:, 1], alpha=0.4, marker='D', s=10)
for n in range(N):
    plt.scatter(X[n, 0], X[n, 1], color=cmap(n), s=50, marker='x', linewidths=2)

データ集合$X$に対応する勝者ノード集合$K^* = \{ k_n^*\}_{n=1}^N$は以下のように求めます．

In [ ]:
print(X.shape, M.shape)
Dist = np.sum((X[:, None, :] - M[None, :, :])**2, axis=2)
print(Dist.shape)

K_star = np.argmin(Dist, axis=1)

K_star

さらに，勝者ノード集合を用いて，データ集合$X$に対応するノード集合$Z = \{ \mathbf{m}_{k_n^*}\}_{n=1}^N$を描画します．

In [ ]:
plt.scatter(M[:, 0], M[:, 1], alpha=0.4, marker='D', s=10)
for n in range(N):
    plt.scatter(X[n, 0], X[n, 1], color=cmap(n), s=50, marker='x', linewidths=2)
    
    z_n = M[K_star[n]]
    
    plt.scatter(z_n[0], z_n[1], marker='D', s=40, color=cmap(n))
#     plt.scatter(M[K_star[n], 0], M[K_star[n], 1], marker='D', s=40, color=cmap(n))

データに一番近いノードの色が，そのデータの色になっていることがわかります（勝者ノードが重複している場合は上書きされます）．

### 4. Fancy index

先ほど，データ集合$X$に対応するノード集合$Z$を求めるのにfor文を用いて実装しました（ひとつ目の方法）．

つまり，

$\forall n, \mathbf{z}_n := \mathbf{m}_{k_n^*}$
`for n in range(N):
    z_n = M[K_ster[n]]`

としました．

これを行列計算（もうひとつの方法）で求める際に用いるのが Fancy Indexです．

イメージとしては以下のような数式になります．  
$Z = M[K^*]$

In [ ]:
Z = M[K_star]
Z, Z.shape, M[0:14,:], M.shape, K_star, len(K_star)

以下，未実装誰かやってくれ
### 5. Einsum
UKRとTSOMの実装にあったら便利なテクニックです．
### 6. Jax
UKRの実装にあったら便利なテクニックです．



## 実装
ここでは，上で得た知識を使って様々な計算を実装していきます．

### I.距離行列


#### I.a 一次元空間
データ$A=\{a_n\}_{n=1}^N$と構造$B=\{b_k\}_{k=1}^K$の要素の間の距離を測れ．

つまり， $D_{ij} = Dist(a_i, b_j)$ のような距離行列(N x K)を求めよ．

*$Dist(a,b)$は$a,b$を入力として実数を返す距離関数．ここでは任意の距離関数を用いて良い．

In [ ]:
N, K = 5, 10
A = np.random.randn(N)
B = np.linspace(-1,1, K)
plt.scatter(A, np.zeros(len(A)), color='r', marker='x')
plt.scatter(B, np.zeros(len(B)), color='b')

#### I.b 二次元空間
データ$A=\{\mathbf{a}_n\}_{n=1}^N, \mathbf{a}\in \mathbb{R}^2$と構造$B=\{\mathbf{b}_k\}_{k=1}^K,\mathbf{b}\in \mathbb{R}^2$の要素の間の距離を測れ．



In [ ]:
N, K = 3, 10
D = 2
A = np.random.randn(N,D).reshape(N,D)
b = np.linspace(-1,1, K)
xx, yy = np.meshgrid(b, b)
B = np.concatenate([xx.reshape(-1)[:, None], yy.reshape(-1)[:, None]], axis=1)
plt.scatter(B[:,0], B[:,1], color='b')
plt.scatter(A[:,0], A[:,1], color='r', marker='x')

#### I.c 任意次元数の空間
データ$A=\{\mathbf{a}_n\}_{n=1}^N, \mathbf{a}\in \mathbb{R}^D$と構造$B=\{\mathbf{b}_k\}_{k=1}^K,\mathbf{b}\in \mathbb{R}^D$の要素の間の距離を測れ．

データと構造を定義するところからはじめよ．処理に任意の次元に対応できるようすればいいだけで，次元数は３でよい．